In [ ]:
#!/usr/bin/env python

*********************************************************************<br>
PROGRAM TO COMPUTE RESIDUALS BETWEEN PARTICLE MOTION ELLIPSES (PMEs)<br>
LITERATURE: Martens et al. (2016, GJI)<br>
<br>
Copyright (c) 2014-2024: HILARY R. MARTENS, LUIS RIVERA, MARK SIMONS         <br>
<br>
This file is part of LoadDef.<br>
<br>
   LoadDef is free software: you can redistribute it and/or modify<br>
   it under the terms of the GNU General Public License as published by<br>
   the Free Software Foundation, either version 3 of the License, or<br>
   any later version.<br>
<br>
   LoadDef is distributed in the hope that it will be useful,<br>
   but WITHOUT ANY WARRANTY; without even the implied warranty of<br>
   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br>
   GNU General Public License for more details.<br>
<br>
   You should have received a copy of the GNU General Public License<br>
   along with LoadDef.  If not, see <https://www.gnu.org/licenses/>.<br>
<br>
*********************************************************************

MODIFY PYTHON PATH TO INCLUDE 'LoadDef' DIRECTORY

In [ ]:
from __future__ import print_function
import sys
import os
sys.path.append(os.getcwd() + "/../../")

IMPORT ADDITIONAL MODULES

In [ ]:
import numpy as np
from utility.pmes import compute_residuals

## USER INPUTS ####

Specify Inputs: Forward Models

In [ ]:
harmonic = "M2"
model1 = "cm_convgf_TPXO9-Atlas_stationMesh_PREM"
model2 = "cm_convgf_FES2014_stationMesh_PREM"
input_directory = ("./output/")
filename1 = (input_directory + "pme_OceanOnly_" + harmonic + "_" + model1 + ".txt")
filename2 = (input_directory + "pme_OceanOnly_" + harmonic + "_" + model2 + ".txt")

Optional: Select Individual Stations to Exclude When Computing the Common-Mode

In [ ]:
stations_to_exclude = []

Filter stations based on latitude?

In [ ]:
filter_lat = False
latitude = 50.

Output filename

In [ ]:
if filter_lat:
    outfile1 = ("Residuals_" + harmonic + "_" +  model1 + "-" + model2 + "_Alaska.txt")
    outfile2 = ("Residuals_" + harmonic + "_" +  model1 + "-" + model2 + "_westUS.txt")
else:
    outfile = ("Residuals_" + harmonic + "_" +  model1 + "-" + model2 + ".txt")

## BEGIN CODE

Set-up Directory

In [ ]:
if not (os.path.isdir("./output/Residuals/")):
    os.makedirs("./output/Residuals/")
outdir = "./output/Residuals/"

Output File

In [ ]:
if filter_lat:
    myoutfile1 = (outdir + outfile1)
    myoutfile2 = (outdir + outfile2)
else:
    myoutfile = (outdir + outfile)

Filter on latitude?

In [ ]:
if filter_lat:

    # Print information
    print(':: Applying a latitude filter to the stations and results.')

    # Identify stations for exclusion based on latitude
    stations = np.loadtxt(filename1,skiprows=1,unpack=True,usecols=(0,),dtype='U')
    slat = np.loadtxt(filename1,skiprows=1,unpack=True,usecols=(1,))
    stations_to_exclude_south = stations_to_exclude[:]
    stations_to_exclude_north = stations_to_exclude[:]
    idx_to_exclude_south = np.where(slat < latitude)[0]
    stations_to_exclude_south.extend(stations[idx_to_exclude_south].tolist())
    idx_to_exclude_north = np.where(slat >= latitude)[0]
    stations_to_exclude_north.extend(stations[idx_to_exclude_north].tolist())

    # Compute Residuals and Remove Common Mode
    rmCMode = True
    compute_residuals.main(filename1,filename2,myoutfile1,rmCMode,stations_to_exclude_south)

    # Compute Residuals but do not Remove Common Mode
    rmCMode = False
    compute_residuals.main(filename1,filename2,myoutfile1,rmCMode,stations_to_exclude_south)

    # Compute Residuals and Remove Common Mode
    rmCMode = True
    compute_residuals.main(filename1,filename2,myoutfile2,rmCMode,stations_to_exclude_north)

    # Compute Residuals but do not Remove Common Mode
    rmCMode = False
    compute_residuals.main(filename1,filename2,myoutfile2,rmCMode,stations_to_exclude_north)